Following thread would run on the client. Broadly, I try to accomplish following sub-tasks. 

1. Create a twitter developer account, get Keys and use twitter developer API for python ( [tweepy](https://www.tweepy.org)) to get real-time tweets.
2. Filter the tweets according to keywords/languages that we are interested in. (Other fields can also be added - Refer to tweepy filter method)
3. Socket programming: Create a socket, and wait for server to connect to the socket, once the connection is established start sending real-time tweets in batches of 1second. (refer to [socket programming](https://docs.python.org/2/howto/sockets.html))





In [0]:
import socket
import sys
import requests
import requests_oauthlib
import json
import tweepy
import matplotlib
import scipy
import numpy
import pandas
import psycopg2
import textblob
from sqlalchemy.exc import ProgrammingError
import json
import string

#Assign a port number for socket
PORT_NO = 9205

#Twitter developer API keys : 

ACCESS_TOKEN = 'bI3xgmRfFLUbbTafxXCeVni6P'
ACCESS_SECRET = 'qO7qUWBB4FjmzCDR9MFlgK1auaPDTmCpMnNXX6hdca9wwsJp2W'
CONSUMER_KEY = '1154929954250874880-5qBMn5m2NVYRNZ18SZKGsxUQ2A7WvL'
CONSUMER_SECRET = 'C6gSAf4qxoAguFJxICvfDwQ0W1Zy552e1Yg5hE9sQtgLl'


#Entities we are interested in : 

TRACK_TERMS = ["Apple", "samsung", "qualcomm", "iphone", "galaxy", "android"]



def filter_ascii(s) : 
  printable = set(string.printable)
  return(filter(lambda x: x in printable, s))

#Override StreamListener method : Here, we can specify stuff to do upon receiving tweets. 

class StreamListener(tweepy.StreamListener):

 
    def on_status(self, status):
      
       #Don't send retweets for processing. Sending the same tweet again and again to spark server which does sentimental analysis is not gonna help ! 
        
        if status.retweeted:
            return
        
        #tweet fields , we are only interested in text and may be location
        
        description = status.user.description
        loc = status.user.location
        text = status.text
        print('---------------------------------------------------------------')
        print(text) 

        #coords = status.coordinates
        #geo = status.geo
        #name = status.user.screen_name
        #user_created = status.user.created_at
        #followers = status.user.followers_count
        #id_str = status.id_str
        #created = status.created_at
        #retweets = status.retweet_count
        #bg_color = status.user.profile_background_color
        #blob = TextBlob(text)
        #sent = blob.sentiment
        #if geo is not None:
        #geo = json.dumps(geo)

        try:
    
          print('Status Obtained')
        
          if text:
            
                # Self explanatory 
                print('sending text to spark master')
                conn.sendall(filter_ascii(text))
          else:
                print('ERROR : No data updte from twitter ')
                                
        except ProgrammingError as err:
            print(err)

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False


# keys Authentication        
          
auth = tweepy.OAuthHandler(ACCESS_TOKEN, ACCESS_SECRET)
auth.set_access_token(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth)
print("API Authentication Complete : ")
print(api)
print('\n Starting a socket \n')


# Start a socket , wait for connection 

host = socket.gethostname()                           
print("hostname = "+host)
server_address = (host.strip(), PORT_NO)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# SO_REUSEADDR field is important since it allows us to reuse the socket. No need to specify new PORT_NO every time you run this thread

s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(server_address)
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.\n Calling Steam Listener")
          
#Call stream listener - send data obtained from twitter to spark master 
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
#print("after stream object")

#Filter tweets - Language is english, Track terms are entities that we are interested in. 
stream.filter(languages=["en"], track = TRACK_TERMS)



API Authentication Complete : 

 Starting a socket 

hostname = b0eee369c1ec
Waiting for TCP connection...
Connected... Starting getting tweets.
 Calling Steam Listener
---------------------------------------------------------------
Me watching Guardians of the Galaxy 2 for the 5th time @JamesGunn @seangunn https://t.co/lcLnn5EB5Y
Status Obtained
sending text to spark master
---------------------------------------------------------------
19 dead, dozens injured in El Paso shooting; 1 suspect arrested, officials say - The Washington Post https://t.co/ImGzhx60Em
Status Obtained
sending text to spark master
---------------------------------------------------------------
RT @xxKarli_xx: The difference between android users and apple users https://t.co/C7owoRAXOq
Status Obtained
sending text to spark master
---------------------------------------------------------------
RT @MichaelSteele: Have you read the #MuellerReport? Don’t worry, @CheriJacobus is bringing the report direct to you with 

KeyboardInterrupt: ignored